# Calculate NPV

## Setup

In [66]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

In [68]:
# Import libraries
import pandas as pd
import numpy as np
from investing.config.file_paths import input_data, output_data
from investing.tools.functions import npv_calc, pv_full_output

<IPython.core.display.Javascript object>

## Configure

In [69]:
# File names
parameters_file = "params_regular.csv"
fcf_start_values = 'fcf_start_data.csv'
npv_file = "npv.csv"

<IPython.core.display.Javascript object>

## Calculate NPV

In [70]:
# Load dataframe of all input values
df_params = pd.read_csv(input_data/parameters_file)
df_fcf_start = pd.read_csv(output_data/ fcf_start_values)

<IPython.core.display.Javascript object>

In [71]:
# Build the dataframe
df = pd.merge(df_params, df_fcf_start,'outer','ticker')

<IPython.core.display.Javascript object>

In [72]:
# Set the Discount Rate
df['discount_rate'] = 10

<IPython.core.display.Javascript object>

In [73]:
# Trim dataframe for example
df = df.iloc[[0]]
df

,ticker,name,type,p10_stage1,p10_stage2,p10_terminal,p50_stage1,p50_stage2,p50_terminal,p90_stage1,p90_stage2,p90_terminal,start_value,discount_rate
0,ACN,Accenture,Regular,10,8,6,8,6,4,6,4,2,13.23,10


<IPython.core.display.Javascript object>

In [74]:
# Calculate the NPV
for pvalue in ['p10','p50','p90']:
    starting_value = df['start_value']
    discount_rate = df['discount_rate']
    perpetual_rate = df[f'{pvalue}_terminal']
    growth_stage1 = df[f'{pvalue}_stage1']
    growth_stage2 = df[f'{pvalue}_stage2']
    
    df[f'npv_{pvalue}'] = np.vectorize(npv_calc)(starting_value, discount_rate, growth_stage1, growth_stage2, perpetual_rate)

<IPython.core.display.Javascript object>

In [75]:
# Calculate the present value forecasts
pv_forecasts = {}
temp_dict = {}
for index, row in df.iterrows():
    ticker = row['ticker']
    for pvalue in ['p10','p50','p90']:
        starting_value = row['start_value']
        discount_rate = row['discount_rate']
        perpetual_rate = row[f'{pvalue}_terminal']
        growth_stage1 = row[f'{pvalue}_stage1']
        growth_stage2 = row[f'{pvalue}_stage2']

        # Set the PV forecast to a temporary dictionary
        temp = pv_full_output(starting_value, discount_rate, growth_stage1, growth_stage2, perpetual_rate)
        # Extract the PV forecast dataframe from the dictionary and set to the temparory dict
        temp_dict[pvalue] = temp['full_pv']
    # Set temp dict to the ticker
    pv_forecasts[ticker] = temp_dict

<IPython.core.display.Javascript object>

In [76]:
pv_forecasts['ACN']['p50']

,years,value,PV
0,1.0,14.288400,12.989455
1,2.0,15.431472,12.753283
2,3.0,16.665990,12.521405
3,4.0,17.999269,12.293743
4,5.0,19.439210,12.070220
5,6.0,20.605563,11.631303
6,7.0,21.841897,11.208347
7,8.0,23.152411,10.800770
8,9.0,24.541555,10.408015
9,10.0,26.014049,10.029542


<IPython.core.display.Javascript object>

In [77]:
# Calculate Risked NPV
df['risked_npv'] = 0.3 * df['npv_p10'] + 0.4 * df['npv_p50'] + 0.3 * df['npv_p90']

<IPython.core.display.Javascript object>

## Export Data

In [78]:
df.to_csv(output_data/npv_file)

<IPython.core.display.Javascript object>